[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/uwsampl/tutorial/blob/master/notebook/tune_relay_cuda.py.ipynb)


Auto-tuning a convolutional network for NVIDIA GPU
==================================================
**Author**: `Lianmin Zheng <https://github.com/merrymercy>`_, `Eddie Yan <https://github.com/eqy/>`_

Auto-tuning for specific devices and workloads is critical for getting the
best performance. This is a tutorial on how to tune a whole convolutional
network for NVIDIA GPU and an extension of the previous tutorial, which focused on tuning a single operator for the GPU.

The operator implementation for NVIDIA GPU in TVM is written in template form.
Whereas the previous tutorial showed how to write a template, in this tutorial we will leverage previously defined templates in TVM's operator inventori (TOPI).
As we have seen, each template has many tunable knobs (tile factor, unrolling, etc).
We will tune all convolution and depthwise convolution operators
in the neural network. After tuning, we produce a log file which stores
the best knob values for all required operators. When the tvm compiler compiles
these operators, it will query this log file to get the best knob values.

We also released pre-tuned parameters for some NVIDIA GPUs. You can go to
[NVIDIA GPU Benchmark](https://github.com/dmlc/tvm/wiki/Benchmark#nvidia-gpu)
to see the results.

Please run the following block to ensure TVM is setup for *this notebook*, each notebook may have its own runtime.

In [1]:
! gsutil cp "gs://tvm-fcrc-binariesd5fce43e-8373-11e9-bfb6-0242ac1c0002/tvm.tar.gz" /tmp/tvm.tar.gz
! mkdir -p /tvm
! tar -xf /tmp/tvm.tar.gz --strip-components=4 --directory /tvm
! ls -la /tvm
# Move this block after we are done with pkg step
! bash /tvm/package.sh
import sys
sys.path.append('/tvm/python')
sys.path.append('/tvm/topi/python')


Copying gs://tvm-fcrc-binariesd5fce43e-8373-11e9-bfb6-0242ac1c0002/tvm.tar.gz...
- [1 files][114.1 MiB/114.1 MiB]                                                
Operation completed over 1 objects/114.1 MiB.                                    
total 164
drwxr-xr-x 21 root root  4096 Jun 14 00:13 .
drwxr-xr-x  1 root root  4096 Jun 14 00:13 ..
drwx------  8 root root  4096 May 31 08:14 3rdparty
drwx------ 12 root root  4096 May 31 08:14 apps
drwx------  3 root root  4096 Jun 11 02:29 build
drwx------  4 root root  4096 Jun 11 02:03 cmake
-rw-------  1 root root 10486 Jun 11 02:03 CMakeLists.txt
drwx------  6 root root  4096 Jun 11 02:03 conda
-rw-------  1 root root  5673 May 31 08:14 CONTRIBUTORS.md
drwx------  3 root root  4096 Jun 11 02:04 docker
drwx------ 11 root root  4096 May 31 08:14 docs
drwx------  4 root root  4096 May 31 08:14 golang
drwx------  3 root root  4096 May 31 08:14 include
-rw-------  1 root root 10243 Jun 11 02:03 Jenkinsfile
drwx------  6 root root  4096 May 31 

Starting a Tracker
---------------------
We start an RPC tracker as in the previous tutorial to manage hardware resources for tuning.

In [2]:
%%script bash --bg --out output --err error
PYTHONPATH=/tvm/python:$PYTHONPATH && python3 -m tvm.exec.rpc_tracker --host 0.0.0.0 --port 9190 &
while true; do
  res=$(PYTHONPATH=/tvm/python:$PYTHONPATH && python3 -m tvm.exec.query_rpc_tracker --host 0.0.0.0 --port 9190 2>&1 | grep 'Cannot connect to tracker')
  if [ "$res" == "" ]; then
    echo "OK @ " $(date) "..." >> status.log
  else
    echo "RESTARTING @ " $(date) "..." >> status.log
    PYTHONPATH=/tvm/python:$PYTHONPATH && python3 -m tvm.exec.rpc_tracker --host 0.0.0.0 --port 9190 &
  fi
  sleep 5
done

Starting job # 0 in a separate thread.


Starting an RPC Server
-------------------------------------
We stat an RPC server as in the previous tutorial to profile schedule configurations during tuning (in this case the GPU associated with the colab runtime), and connect it to the tracker.

In [3]:
%%script bash --bg --out output2 --err error2
while true; do
echo "started server at " $(date) >> status.log
PYTHONPATH=/tvm/python:/tvm/topi/python:$PYTHONPATH && python3 -m tvm.exec.rpc_server --key 1080ti --tracker 0.0.0.0:9190
sleep 30
done


Starting job # 2 in a separate thread.


Check the status of the Tracker and Server
--------------------------------------------------------------------
We check that there is a free GPU device reported by the tracker before continuing.

In [4]:
! cat status.log | tail
! PYTHONPATH=/tvm/python:$PYTHONPATH && python3 -m tvm.exec.query_rpc_tracker --host 0.0.0.0 --port 9190 

started server at  Fri Jun 14 00:21:21 UTC 2019
Tracker address 0.0.0.0:9190

Server List
----------------------------
server-address	key
----------------------------
127.0.0.1:54028	server:1080ti
----------------------------

Queue Status
------------------------------
key      total  free  pending
------------------------------
1080ti   1      1     0      
------------------------------



Import required python modules:

In [0]:
import os
import numpy as np
import tvm
from tvm import autotvm
from tvm import relay
import tvm.relay.testing
from tvm.autotvm.tuner import XGBTuner, GATuner, RandomTuner, GridSearchTuner
from tvm.contrib.util import tempdir
import tvm.contrib.graph_runtime as runtime

 Define Network
 --------------
First we need to define the network in relay frontend API.
We can load some pre-defined network from :code:`nnvm.testing`.
We can also load models from MXNet, ONNX and TensorFlow.




In [0]:
def get_network(name, batch_size):
    """Get the symbol definition and random weight of a network"""
    input_shape = (batch_size, 3, 224, 224)
    output_shape = (batch_size, 1000)

    if "resnet" in name:
        n_layer = int(name.split('-')[1])
        net, params = relay.testing.resnet.get_workload(num_layers=n_layer, batch_size=batch_size, dtype=dtype)
    elif "vgg" in name:
        n_layer = int(name.split('-')[1])
        net, params = relay.testing.vgg.get_workload(num_layers=n_layer, batch_size=batch_size, dtype=dtype)
    elif name == 'mobilenet':
        net, params = relay.testing.mobilenet.get_workload(batch_size=batch_size, dtype=dtype)
    elif name == 'squeezenet_v1.1':
        net, params = relay.testing.squeezenet.get_workload(batch_size=batch_size, version='1.1', dtype=dtype)
    elif name == 'inception_v3':
        input_shape = (1, 3, 299, 299)
        net, params = relay.testing.inception_v3.get_workload(batch_size=batch_size, dtype=dtype)
    elif name == 'mxnet':
        # an example for mxnet model
        from mxnet.gluon.model_zoo.vision import get_model
        block = get_model('resnet18_v1', pretrained=True)
        net, params = relay.frontend.from_mxnet(block, shape={'data': input_shape}, dtype=dtype)
        net = relay.Function(net.params, relay.nn.softmax(net.body), None, net.type_params, net.attrs)
    else:
        raise ValueError("Unsupported network: " + name)

    return net, params, input_shape, output_shape



Set Tuning Options
 ------------------
Before tuning, we apply some configuration options. Thse include the target device (CUDA GPU), the selected network, paths for logging, and tuning parameters. The tuning parameters specify which tuner to use and how many trials (measurement on hardware) to perform before moving to the next tuning task.

These parameters are set given the constraints of the colab runtime (CPU resources) and stability of long running jobs and for demo purposes. We recommend running full-scale experiments on dedicated hardware.
If settings with larger time budgets,  set :code:`n_trial`, :code:`early_stopping` larger, which makes the tuning runs longer.
If you have multiple devices, you can use all of them for measurement to accelerate the tuning process. (see the 'Scale up measurement` section below).

In [0]:
#### DEVICE CONFIG ####
target = tvm.target.cuda()

#### TUNING OPTION ####
network = 'resnet-18'
log_file = "%s.log" % network
dtype = 'float32'

tuning_option = {
    'log_filename': log_file,

    'tuner': 'xgb',
    'n_trial': 128,
    'early_stopping': 32,

    'measure_option': autotvm.measure_option(
        builder=autotvm.LocalBuilder(timeout=2),
        #runner=autotvm.LocalRunner(number=20, repeat=3, timeout=4, min_repeat_ms=150),
        runner=autotvm.RPCRunner(
            '1080ti',  # change the device key to your key
            '0.0.0.0', 9190,
            number=20, repeat=3, timeout=1, min_repeat_ms=50)
    ),
}



Begin Tuning
------------
Now we can extract tuning tasks from the network and begin tuning.
Here, we provide a simple utility function to tune a list of tasks.
This function is just an initial implementation which tunes them in sequential order.
We will introduce a more sophisticated tuning scheduler in the future.

In [0]:
# You can skip the implementation of this function for this tutorial.
def tune_tasks(tasks,
               measure_option,
               tuner='xgb',
               n_trial=1000,
               early_stopping=None,
               log_filename='tuning.log',
               use_transfer_learning=True,
               try_winograd=True):
    if try_winograd:
        for i in range(len(tasks)):
            try:  # try winograd template
                tsk = autotvm.task.create(tasks[i].name, tasks[i].args,
                                          tasks[i].target, tasks[i].target_host, 'winograd')
                input_channel = tsk.workload[1][1]
                if input_channel >= 64:
                    tasks[i] = tsk
            except Exception:
                pass

    # create tmp log file
    tmp_log_file = log_filename + ".tmp"
    if os.path.exists(tmp_log_file):
        os.remove(tmp_log_file)

    for i, tsk in enumerate(reversed(tasks)):
        prefix = "[Task %2d/%2d] " %(i+1, len(tasks))

        # create tuner
        if tuner == 'xgb' or tuner == 'xgb-rank':
            tuner_obj = XGBTuner(tsk, feature_type='knob', loss_type='rank')
        elif tuner == 'ga':
            tuner_obj = GATuner(tsk, pop_size=100)
        elif tuner == 'random':
            tuner_obj = RandomTuner(tsk)
        elif tuner == 'gridsearch':
            tuner_obj = GridSearchTuner(tsk)
        else:
            raise ValueError("Invalid tuner: " + tuner)

        if use_transfer_learning:
            if os.path.isfile(tmp_log_file):
                tuner_obj.load_history(autotvm.record.load_from_file(tmp_log_file))

        # do tuning
        tuner_obj.tune(n_trial=min(n_trial, len(tsk.config_space)),
                       early_stopping=early_stopping,
                       measure_option=measure_option,
                       callbacks=[
                           autotvm.callback.progress_bar(n_trial, prefix=prefix),
                           autotvm.callback.log_to_file(tmp_log_file)])

    # pick best records to a cache file
    autotvm.record.pick_best(tmp_log_file, log_filename)
    os.remove(tmp_log_file)

Next, we launch tuning jobs and evaluate the end-to-end performance.
`tune_and_evaluate` extracts the tuning tasks (the operators/workloads to tune) from the specified network and tunes them.
After tuning, `autotvm.apply_history_best` applies the best configurations logged during tuning and evaluates the end-to-end performance achieved when running the entire model using tuned operators.

In [0]:
def tune_and_evaluate(tuning_opt):
    # extract workloads from relay program
    print("Extract tasks...")
    net, params, input_shape, out_shape = get_network(network, batch_size=1)
    tasks = autotvm.task.extract_from_program(net, target=target,
                                            params=params, ops=(relay.op.nn.conv2d,))

    # run tuning tasks
    print("Tuning...")
    tune_tasks(tasks, **tuning_opt)

    # compile kernels with history best records
    with autotvm.apply_history_best(log_file):
        print("Compile...")
        with relay.build_config(opt_level=3):
            graph, lib, params = relay.build_module.build(
                net, target=target, params=params)

        # export library
        tmp = tempdir()
        filename = "net.tar"
        lib.export_library(tmp.relpath(filename))

        # load parameters
        ctx = tvm.context(str(target), 0)
        module = runtime.create(graph, lib, ctx)
        data_tvm = tvm.nd.array((np.random.uniform(size=input_shape)).astype(dtype))
        module.set_input('data', data_tvm)
        module.set_input(**params)

        # evaluate
        print("Evaluate inference time cost...")
        ftimer = module.module.time_evaluator("run", ctx, number=1, repeat=600)
        prof_res = np.array(ftimer().results) * 1000  # convert to millisecond
        print("Mean inference time (std dev): %.2f ms (%.2f ms)" %
              (np.mean(prof_res), np.std(prof_res)))

tune_and_evaluate(tuning_option)

#################################################################
# Scale up measurement by using multiple devices
# ----------------------------------------------
#
# If you have multiple devices, you can use all of them for measurement.
# TVM uses the RPC Tracker to manage distributed devices.
# The RPC Tracker is a centralized master node. We can register all devices to
# the tracker. For example, if we have 10 GPU cards, we can register all of them
# to the tracker, and run 10 measurements in parallel, accelerating the tuning process.
#
# To start an RPC tracker, run this command on the host machine. The tracker is
# required during the whole tuning process, so we need to open a new terminal for
# this command:
#
# .. code-block:: bash
#
#   python -m tvm.exec.rpc_tracker --host=0.0.0.0 --port=9190
#
# The expected output is
#
# .. code-block:: bash
#
#   INFO:RPCTracker:bind to 0.0.0.0:9190
#
# Then open another new terminal for the RPC server. We need to start one server
# for each dedicated device. We use a string key to distinguish the types of devices.
# You can pick a name you like.
# (Note: For rocm backend, there are some internal errors with the compiler,
# we need to add `--no-fork` to the argument list.)
#
# .. code-block:: bash
#
#     python -m tvm.exec.rpc_server --tracker=0.0.0.0:9190 --key=1080ti
#
# After registering devices, we can confirm it by querying rpc_tracker
#
# .. code-block:: bash
#
#   python -m tvm.exec.query_rpc_tracker --host=0.0.0.0 --port=9190
#
# For example, if we have four 1080ti, two titanx and one gfx900, the output can be
#
# .. code-block:: bash
#
#    Queue Status
#    ----------------------------------
#    key          total  free  pending
#    ----------------------------------
#    1080ti       4      4     0
#    titanx       2      2     0
#    gfx900       1      1     0
#    ----------------------------------
#
# Finally, we need to change the tuning option to use RPCRunner. Use the code below
# to replace the corresponding part above.

tuning_option = {
    'log_filename': log_file,

    'tuner': 'gatuner',
    'n_trial': 2000,
    'early_stopping': 200,

    'measure_option': autotvm.measure_option(
        builder=autotvm.LocalBuilder(timeout=10),
        runner=autotvm.RPCRunner(
            '1080ti',  # change the device key to your key
            '0.0.0.0', 9190,
            number=20, repeat=3, timeout=2, min_repeat_ms=50),
    ),
}

Extract tasks...
...100%, 0.02 MB, 104 KB/s, 0 seconds passed
Tuning...
[Task  1/12]  Current/Best:  143.35/ 258.37 GFLOPS | Progress: (44/128) | 43.04 s Done.
[Task  2/12]  Current/Best:  150.26/ 672.24 GFLOPS | Progress: (80/128) | 97.66 s Done.
[Task  3/12]  Current/Best:   23.25/ 298.16 GFLOPS | Progress: (38/128) | 38.68 s Done.
[Task  4/12]  Current/Best:    0.00/ 766.28 GFLOPS | Progress: (38/128) | 37.69 s Done.
[Task  5/12]  Current/Best:    0.00/1419.15 GFLOPS | Progress: (52/128) | 46.29 s Done.
[Task  6/12]  Current/Best: 1329.87/2709.25 GFLOPS | Progress: (38/128) | 50.15 s Done.
[Task  7/12]  Current/Best:    0.00/ 931.21 GFLOPS | Progress: (64/128) | 69.56 s